#  **C4 Diel leaf model**

This file discussesmaking C3 model from PlantCoreMetabolism model by applying required constraints

#Step:1  Install cobrapy
* Run `%pip install cobra` to install cobrapy

In [ ]:
%pip install cobra  --quiet

#Step:2 Import C4_model from Github
* Use wget using the syntax `!wget link-to-file` to retrieve the models and functions required for this analysis


In [ ]:
# Retrieving the model
!wget -q https://github.com/sshameer/DielLeafModeling/raw/refs/heads/main/Models/C4_model.sbml

# Step: 3 Import functions required to read sbml and perform pFBA
* A cobra model can be generated from an SBML file using the` read_sbml_model` function in the cobrapy io module
* pFBA can be performed using the pfba package in the cobrapy `flux_analysis` module

In [ ]:
from cobra.io import read_sbml_model
from cobra.flux_analysis import pfba

#Step:4 Import the model and constraining the model for diel leaf model of C4 photosynthesis
* The syntax for loading the model is `model_name = read_sbml_model(file_name)`

In [ ]:
C4_model = read_sbml_model("C4_model.sbml")

* setting the PPFD and photon uptake as 500 for mesophyll and and bundle sheath
* Defining the maintenance cost separately for mesophyll and bundle sheath as by referring [Töpfer et al.,2020](https://doi.org/10.1105/tpc.20.00132)

In [ ]:
# Setting photon uptake
BS_PPFD = 500
M_PPFD = 500
C4_model.reactions.Photon_tx1.upper_bound = M_PPFD
C4_model.reactions.Photon_tx3.upper_bound = BS_PPFD

#Defining maintenance cost in bundle sheath

BS_VATPase = 0.0049*BS_PPFD+2.7851
C4_model.reactions.get_by_id("ATPase_tx3").bounds = (BS_VATPase,BS_VATPase)

#Defining maintenance cost in Mesophyll

M_VATPase = 0.0049*M_PPFD+2.7851
C4_model.reactions.get_by_id("ATPase_tx1").bounds=(M_VATPase,M_VATPase)



In [ ]:
# Restrict mesophyll sucrose, mesophyll starch and bundlesheath sucrose accumulation
for id in ["SUCROSE_v_dielTransfer_BS","SUCROSE_v_dielTransfer_M","STARCH_p_dielTransfer_M"]:
    rxn = C4_model.reactions.get_by_id(id)
    rxn.upper_bound = 0
    rxn.lower_bound = 0

# Step:5 get the summary of the model after optimization
Run the syntax `model_name.summary()` for getting the summary of the model

In [ ]:
C4sol = pfba(C4_model)
C4_model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
CARBON_DIOXIDE_e1,CO2_tx1,94.03,1,100.00%
WATER_e1,H2O_tx1,96.25,0,0.00%
NITRATE_e1,Nitrate_tx1,0.5877,0,0.00%
NITRATE_e4,Nitrate_tx4,0.3918,0,0.00%
OXYGEN_MOLECULE_e2,O2_tx2,2.787,0,0.00%
OXYGEN_MOLECULE_e4,O2_tx4,1.329,0,0.00%
Photon_e1,Photon_tx1,500,0,0.00%
Photon_e3,Photon_tx3,500,0,0.00%
SULFATE_e1,SO4_tx1,0.007489,0,0.00%
PROTON_c1,unlProtHYPO_c1,0.853,0,0.00%


#Step:6 Confirming the C4 diel model for PEPCK C4 photosynthesis
for the confirmation checked the transfer reactions between mesophyll and  bundle sheath

In [ ]:
for rxn in C4_model.reactions.query("_MBStransfer_Day"):
  if abs(rxn.flux)>20:
    print(rxn.id+"\t"+rxn.reaction+"\t"+str(rxn.flux))

GAP_c_MBStransfer_Day	GAP_c1 <=> GAP_c3	77.97396297005727
MAL_c_MBStransfer_Day	MAL_c1 <=> MAL_c3	39.645652981156914
G3P_c_MBStransfer_Day	G3P_c1 <=> G3P_c3	-78.11796842381614
Pi_c_MBStransfer_Day	Pi_c1 <=> Pi_c3	38.81087010673567
L_ALPHA_ALANINE_c_MBStransfer_Day	L_ALPHA_ALANINE_c1 <=> L_ALPHA_ALANINE_c3	-48.764133119963134
PHOSPHO_ENOL_PYRUVATE_c_MBStransfer_Day	PHOSPHO_ENOL_PYRUVATE_c1 <=> PHOSPHO_ENOL_PYRUVATE_c3	-55.30009469872094
L_ASPARTATE_c_MBStransfer_Day	L_ASPARTATE_c1 <=> L_ASPARTATE_c3	54.294964752150904


When observing fluxes between mesophyll(M) and bundle sheath(BS) during the day, we see Aspartate, malate and triose-P transfer from M to BS. ALA, PEP and 3PG can also be observed to move from BS to M, indicative of C4 PEPCK.

In [ ]:
# creating a copy for making other types of C4 pathways
backup = C4_model.copy()

#Step:7  Configuring the C4 diel model for NAD-ME C4 photosynthesis
* As observed earlier, the default configuration of the diel C4 model is PEPCK C4 photosynthesis.
* In order to configure the model for NAD-ME C4, where mitochondrial Malic Enzyme (NAD-ME) is the primary decarboxylating enzyme in the bundle sheath, one need to constrain NADP-ME and PEPCK reactions to carry zero flux


In [ ]:
#Making a copy of backup to convert the model into NAD-ME type
C4_model = backup.copy()

#constrain bundle sheath NADP-ME to 0
C4_model.reactions.get_by_id("MALIC_NADP_RXN_p3").lower_bound = 0
C4_model.reactions.get_by_id("MALIC_NADP_RXN_p3").upper_bound = 0
C4_model.reactions.get_by_id("MALIC_NADP_RXN_c3").lower_bound = 0
C4_model.reactions.get_by_id("MALIC_NADP_RXN_c3").upper_bound = 0

#constrain bundle sheath PEPCK to 0
C4_model.reactions.get_by_id("PEPCARBOXYKIN_RXN_c3").lower_bound = 0
C4_model.reactions.get_by_id("PEPCARBOXYKIN_RXN_c3").upper_bound = 0

#ensure bundle sheath NAD-ME is active
C4_model.reactions.get_by_id("1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m3").lower_bound = 0
C4_model.reactions.get_by_id("1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m3").upper_bound = 1000

C4sol2 = pfba(C4_model)
C4_model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
CARBON_DIOXIDE_e1,CO2_tx1,89.29,1,100.00%
WATER_e1,H2O_tx1,80.21,0,0.00%
WATER_e4,H2O_tx4,0.7114,0,0.00%
NITRATE_e3,Nitrate_tx3,0.5573,0,0.00%
NITRATE_e4,Nitrate_tx4,0.3715,0,0.00%
OXYGEN_MOLECULE_e2,O2_tx2,2.998,0,0.00%
OXYGEN_MOLECULE_e4,O2_tx4,1.146,0,0.00%
Photon_e1,Photon_tx1,500,0,0.00%
Photon_e3,Photon_tx3,500,0,0.00%
SULFATE_e1,SO4_tx1,0.003551,0,0.00%


#Step:8 Confirming the C4 diel model for NAD-ME C4 photosynthesis

for the confirmation checked the transfer reactions between mesophyll and bundle sheath

In [ ]:
# Check for large metabolite exchange between Mesophyll and Bundle sheath cells
for rxn in C4_model.reactions.query("_MBStransfer_Day"):
  if abs(rxn.flux)>20:
    print(rxn.id+"\t"+rxn.reaction+"\t"+str(rxn.flux))

GAP_c_MBStransfer_Day	GAP_c1 <=> GAP_c3	75.76196964819825
MAL_c_MBStransfer_Day	MAL_c1 <=> MAL_c3	40.456119314344974
G3P_c_MBStransfer_Day	G3P_c1 <=> G3P_c3	-75.07590032639052
aPi_c_MBStransfer_Day	aPi_c1 <=> aPi_c3	11.70207341257748
Pi_c_MBStransfer_Day	Pi_c1 <=> Pi_c3	27.304837962680768
L_ALPHA_ALANINE_c_MBStransfer_Day	L_ALPHA_ALANINE_c1 <=> L_ALPHA_ALANINE_c3	-43.831072420903475
PHOSPHO_ENOL_PYRUVATE_c_MBStransfer_Day	PHOSPHO_ENOL_PYRUVATE_c1 <=> PHOSPHO_ENOL_PYRUVATE_c3	-39.69298069706593
L_ASPARTATE_c_MBStransfer_Day	L_ASPARTATE_c1 <=> L_ASPARTATE_c3	48.75295707978313


In [ ]:
print("Observing PEPC activity ...")
# Check activity of all C4 decarboxylating enzymes
for rxn in ["PEPCARBOX_RXN_c1",]:
  rxn = C4_model.reactions.get_by_id(rxn)
  print(rxn.id+"\t"+rxn.name+"\t"+str(rxn.flux))


print("\n\nObserving activity of decarboxylating enzymes...")
# Check activity of all C4 decarboxylating enzymes
for rxn in ["MALIC_NADP_RXN_p3","MALIC_NADP_RXN_c3","PEPCARBOXYKIN_RXN_c3","1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m3"]:
  rxn = C4_model.reactions.get_by_id(rxn)
  print(rxn.id+"\t"+rxn.name+"\t"+str(rxn.flux))

Observing PEPC activity ...
PEPCARBOX_RXN_c1	PEPCARBOX-RXN:Phosphoenolpyruvate carboxylase M Day	85.79577973705318


Observing activity of decarboxylating enzymes...
MALIC_NADP_RXN_p3	MALIC-NADP-RXN:Malate dehydrogenase (oxaloacetate decarboxylating) (NADP+) BS Day	0.0
MALIC_NADP_RXN_c3	MALIC-NADP-RXN:Malate dehydrogenase (oxaloacetate decarboxylating) (NADP+) BS Day	0.0
PEPCARBOXYKIN_RXN_c3	PEPCARBOXYKIN-RXN:Phosphoenolpyruvate carboxykinase (ATP) BS Day	0.0
1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m3	MALIC-NAD-RXN:Malate dehydrogenase (oxaloacetate decarboxylating) BS Day	85.26261202032642


As you can observe here, with the addition of constraints for NAD-ME, NAD-ME is active decarboxylating enzyme in BS. Triose-P, Aspartate and Malate flux can be observed from M to BS, and 3PG, Alanine and PEP flux can be observed from BS to M. This is indicative of NAD-ME C4 subtype.

#Step:9  Configuring the C4 diel model for NADP-ME C4 photosynthesis

 In order to configure the model for NADP-ME C4, where plastidic Malic Enzyme (NADP-ME) is the primary decarboxylating enzyme in the bundle sheath, one need to constrain NAD-ME and PEPCK reactions to carry zero flux

In [ ]:
#Making a copy of backup to convert the model into NADP-ME type
C4_model = backup.copy()

#ensure bundle sheath plastidic NADP-ME is active
C4_model.reactions.get_by_id("MALIC_NADP_RXN_p3").lower_bound = 0
C4_model.reactions.get_by_id("MALIC_NADP_RXN_p3").upper_bound = 1000
#constrain bundle sheath cytosolic NADP-ME to 0 for simplicity
C4_model.reactions.get_by_id("MALIC_NADP_RXN_c3").lower_bound = 0
C4_model.reactions.get_by_id("MALIC_NADP_RXN_c3").upper_bound = 0

#constrain bundle sheath PEPCK to 0
C4_model.reactions.get_by_id("PEPCARBOXYKIN_RXN_c3").lower_bound = 0
C4_model.reactions.get_by_id("PEPCARBOXYKIN_RXN_c3").upper_bound = 0

#constrain bundle sheath NAD-ME to 0
C4_model.reactions.get_by_id("1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m3").lower_bound = 0
C4_model.reactions.get_by_id("1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m3").upper_bound = 0

# bundle sheath PSII flux has been reported to be absent in some NADP-ME leaves
C4_model.reactions.get_by_id("PSII_RXN_p3").lower_bound = 0
C4_model.reactions.get_by_id("PSII_RXN_p3").upper_bound = 0

C4sol2 = pfba(C4_model)
C4_model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
CARBON_DIOXIDE_e1,CO2_tx1,56.57,1,100.00%
WATER_e1,H2O_tx1,72.87,0,0.00%
NITRATE_e1,Nitrate_tx1,0.3459,0,0.00%
NITRATE_e2,Nitrate_tx2,0.1989,0,0.00%
NITRATE_e4,Nitrate_tx4,0.03169,0,0.00%
OXYGEN_MOLECULE_e2,O2_tx2,3.359,0,0.00%
OXYGEN_MOLECULE_e3,O2_tx3,3.841,0,0.00%
OXYGEN_MOLECULE_e4,O2_tx4,0.2724,0,0.00%
Photon_e1,Photon_tx1,500,0,0.00%
Photon_e3,Photon_tx3,359.8,0,0.00%


#Step:10 Confirming the C4 diel model for NADP-ME C4 photosynthesis

for the confirmation checked the transfer reactions between mesophyll and bundle sheath

In [ ]:
# Check for large metabolite exchange between Mesophyll and Bundle sheath cells
for rxn in C4_model.reactions.query("_MBStransfer_Day"):
  if abs(rxn.flux)>20:
    print(rxn.id+"\t"+rxn.reaction+"\t"+str(rxn.flux))

GAP_c_MBStransfer_Day	GAP_c1 <=> GAP_c3	54.61669903739009
PYRUVATE_c_MBStransfer_Day	PYRUVATE_c1 <=> PYRUVATE_c3	-59.37568406840053
MAL_c_MBStransfer_Day	MAL_c1 <=> MAL_c3	64.41662643185354
G3P_c_MBStransfer_Day	G3P_c1 <=> G3P_c3	-53.79636182111048


In [ ]:
print("Observing PEPC activity ...")
# Check activity of all C4 decarboxylating enzymes
for rxn in ["PEPCARBOX_RXN_c1",]:
  rxn = C4_model.reactions.get_by_id(rxn)
  print(rxn.id+"\t"+rxn.name+"\t"+str(rxn.flux))


print("\n\nObserving activity of decarboxylating enzymes...")
# Check activity of all C4 decarboxylating enzymes
for rxn in ["MALIC_NADP_RXN_p3","MALIC_NADP_RXN_c3","PEPCARBOXYKIN_RXN_c3","1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m3"]:
  rxn = C4_model.reactions.get_by_id(rxn)
  print(rxn.id+"\t"+rxn.name+"\t"+str(rxn.flux))

Observing PEPC activity ...
PEPCARBOX_RXN_c1	PEPCARBOX-RXN:Phosphoenolpyruvate carboxylase M Day	56.570632600035786


Observing activity of decarboxylating enzymes...
MALIC_NADP_RXN_p3	MALIC-NADP-RXN:Malate dehydrogenase (oxaloacetate decarboxylating) (NADP+) BS Day	56.5571844291538
MALIC_NADP_RXN_c3	MALIC-NADP-RXN:Malate dehydrogenase (oxaloacetate decarboxylating) (NADP+) BS Day	0.0
PEPCARBOXYKIN_RXN_c3	PEPCARBOXYKIN-RXN:Phosphoenolpyruvate carboxykinase (ATP) BS Day	0.0
1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m3	MALIC-NAD-RXN:Malate dehydrogenase (oxaloacetate decarboxylating) BS Day	0.0


As you can observe here, with the addition of constraints for NADP-ME, NADP-ME is active decarboxylating enzyme in BS. Triose-P and Malate flux can be observed from M to BS, and 3PG and Pyruvate flux can be observed from BS to M. This is indicative of NADP-ME C4 subtype.

*Note that we have only considered a few critical and large fluxes in this analysis. For a proper analysis, a flux map of the predicted flux distribution should be developed, similar to Figure 3 in the manuscript.*